In [1]:
import sys
import os
from typing import Dict, List

import numpy as np
import tensorflow as tf
print(tf.__version__)

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=80) 

%load_ext line_profiler

2.3.0


# Constants

In [2]:
from preprocessing import (
    DELIMITER,
    SPACE,
    NIL
)

STRIDE = int(input("STRIDE size?"))
CONTEXT_SIZE = 1 + (STRIDE * 2)

USE_PTB = bool(input("USE_PTB? Just enter for False or any string for True"))
USE_NATIVE=True

STRIDE size?2
USE_PTB? Just enter for False or any string for True


In [3]:
DEBUG = False
VALIDATION = True

# Text to process

### Test sample text

In [4]:
#corpus = "The fool doth think he is wise, but the wise man knows himself to be a fool."
#corpus = "To be, or not to be, that is the question"
#corpus = "To to be be, or not not not not not to be, that is that the question that matters"
corpus = "To be, or not to be, that is the question that matters"

#corpus = "I know how to build an attention in neural networks. But I don’t understand how attention layers learn the weights that pay attention to some specific embedding. I have this question because I’m tackling a NLP task using attention layer. I believe it should be very easy to learn (the most important part is to learn alignments). However, my neural networks only achieve 50% test set accuracy. And the attention matrix is weird. I don’t know how to improve my networks."

## PTB (Penn Treebank) 

In [5]:
from data.ptb import load_data

/home/oonisim/dataset


# Utilities

### Padding

To avoid boundary checking when iterate through the sequenced corpus, pad the source text with '<nil>'.
e.g. (when context is of size 5):    
From:
```
|B|X|Y|Z|...|P|Q|R|E|
```

To:
```
|<nil>|<nil>|B|X|Y|Z|...|P|Q|R|E|<nil>|<nil>| 
```

In [1]:
from preprocessing.text import (
    PAD_MODE_SQUEEZE,
    pad_text
)

# Word indexing
Assign a numerical id to each word.

The row index of co-occurrence matrix is a word index. The number of words in the corpus can be less than the number of word indices because additional meta-word such as OOV, UNK, NIL can be added to the original corpus.

Make sure **the co-occurrence matrix row index matches with the word index**, unless explicitly adjust when row-index and word-index do not match.

## Load the corpus text

In [9]:
if USE_PTB:
    corpus = pad_text(
        corpus=load_text('train'), mode=PAD_MODE_SQUEEZE, delimiter=SPACE, padding=NIL, length=STRIDE
    )
else:
    print("Original corpus: \n[{}]".format(corpus))
    corpus = pad_text(
        corpus=corpus, mode=PAD_MODE_SQUEEZE, delimiter=SPACE, padding=NIL, length=STRIDE
    )
    print("Padded corpus: \n[{}]".format(corpus))

Original corpus: 
[To be, or not to be, that is the question that matters]
Padded corpus: 
[<nil> <nil> To be, or not to be, that is the question that matters <nil> <nil>]


## Word indexing

### Native word indexing

In [ ]:
from preprocessing.text import (
    text_to_sequence
)

### Native indexing

In [11]:
corpus = re.sub(r'[.,:;]+', SPACE, corpus.lower())

In [12]:
if USE_NATIVE:
    (sequence, word_to_id, id_to_word, vocabulary_size) = native_word_indexing(corpus)

print(vocabulary_size)
if not USE_PTB:
    print(word_to_id)

9927


### Tensorflow Tokenizer indexing

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")

USE_TENSORFLOW = (not USE_NATIVE)
if USE_TENSORFLOW:
    # Each text in "texts" is a complete document as one string, 
    # e.g "To be or not to be, that is the question."
    texts = [ corpus ]   

    # fit_on_texts() processes multiple documents and handles all words in all the documents.
    tokenizer.fit_on_texts(texts)
    word_to_id = tokenizer.word_index

    # texts_to_sequences() ruturns sequences, one sequence for each text in "texts".
    sequences = (tokenizer.texts_to_sequences(texts))
    sequence = sequences[0]

    print(len(sequences))
    print(len(word_to_id))
    
    # Index of tokenizer.word_index starts at 1, NOT 0.
    # e.g. {'<OOV>': 1, 'the': 2, 'fool': 3, 'wise': 4, 'doth': 5, ...}
    vocabulary_size = max(word_to_id.values()) + 1
    print(vocabulary_size)

## Verification

In [14]:
if not USE_PTB:
    print("word to id \n{}".format(word_to_id))
    print("id to word \n{}".format(id_to_word))
    print()
    print("corpus is \n[{}]".format(corpus))
    print("sequence is \n{}".format(sequence))
    print("corpus size is {} sequence size is {} expected sum is {}".format(
        len(re.compile('[\t\s]+').split(corpus)), 
        len(sequence), 
        (len(sequence) - (2*STRIDE)) * (2*STRIDE)  # Exclude NIL from the sequence
    ))
    #print([id_to_word[index] for index in sequence])
    print(np.array([id_to_word[index] for index in sequence]))

# Co-occurrence Matrix

In [3]:
from np.analytics import (
    create_cooccurrence_matrix,
    cooccurrence_words,
    word_frequency,
    total_frequencies
)

In [ ]:
matrix = create_cooccurrence_matrix(sequence, vocabulary_size, CONTEXT_SIZE)

if VALIDATION:
    print(matrix.shape)

    if not USE_PTB:
        print(matrix)
        print("com.sum() {}".format(matrix.sum()))

    assert total_frequencies(matrix, word_to_id, CONTEXT_SIZE, NIL) == len(sequence) - (CONTEXT_SIZE -1)
    assert np.array_equal(com0, com)

# PPMI